# ResNet (CNN)
* VGG : 3 x 3 합성곱 -> 커널 -> 필터 ...
* 층을 많이 쌓으면 기울이가 0이 된다 = 기울기 소실 (0~1)
* VGG -> 19층. / 층 -> 특징

**ResNet** : 스킵 커넥션 -> 100층까지...

층이 많다고 해서? 다다익선?
- 너무 많은 층을 가져가면 한 픽셀의 영향도가 너무 커진다
- 합성곱 => 특정 점으로 압축 => 너무 많은 층? => 너무 많은 압축
(분류? -> 불이익)

층이 많아지면 -> 기울기가 소실된다 -> 가중치가 잘 업데이트 안 된다

스킵 커넥션 (**Skip Connection**)
- 자기 자신을 미분하고 신경망의 출력 부분에 입력을 더하는 방식으로 기울기를 최소 1로 확보하는 기법

---
ResNet

|장점|단점|
|-|-|
|층을 깊게 쌓을 수 있다|가중치가 늘어난다 -> 계산량이 많아짐|
|VGG에 비해 학습이 안정적이다|VGG에 비해서 오버피팅이 일어나기 쉽다|
|기울기 소실 문제가 어느정도 해결| |

## 스킵 커넥션 <- nn.Module (복잡-흐름제어) / nn.Sequential (단순)
* (합성곱 3x3 -> 배치 정규화 -> ReLU -> 합성곱 3x3 -> 배치정규화 -> ... -> ReLU)
### 배치 정규화
    * 딥러닝 층이 깊어지다보면 각 층마다 값의 범위가 달라지는 경우 <- 배치(Batch) 단위로 데이터를 학습. 배치 그 안의 데이터 차이 때문에
    * 배치 정규화 (Batch Normalization) : 배치 간의 데이터 분포 불균형을 해결하는 기법 -> 각 딥러닝 모델 층 - 값의 분포가 일정해지게 됨

In [ ]:
#@title ResNet 기본 블록
import torch
import torch.nn as nn

In [ ]:
class BasicBlock(nn.Module): # Module 상속
    # 생성할 때 3개의 패러미터
    # in_channels : 입력층 갯수
    # out_channels : 출력층 갯수
    # kernel_size : 합성곱의 사이즈
    def __init__(self, in_channels, out_channels,
                #  hidden_dim, 은닉층에서 어떻게 채널을 전달할지?
                 kernel_size=3):
        super(BasicBlock, self).__init__()

        # 합성곱층 정의
        self.c1 = nn.Conv2d(in_channels, out_channels,
                            kernel_size=kernel_size, padding=1)
        self.c2 = nn.Conv2d(out_channels, out_channels,
                            kernel_size=kernel_size, padding=1)
        # 다운샘플 : 스킵커넥션 - 입력값과 합성곱의 결과(은닉층)를 더해주는 과정
        # 이미지는 채널(R,G,B) 3개 입력값 / 특징 맵 -> 64, 256... 
        # 입력값에 1x1 합성곱으로 채널 수를 맞춰주는 기법
        self.downsample = nn.Conv2d(
            in_channels, out_channels, kernel_size=1)
        
        # 배치 정규화 - 이미지의 출력되는 채널 수
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)

        self.relu = nn.ReLU()
    
    # 순전파 정의
    def forward(self, x):
        # 스킵 커넥션 -> 최종적으로 처리된 결과에 입력값을 더해줌
        x_ = x # 초기 입력

        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)

        x_ = self.downsample(x_)

        x += x_ # <- 기울기 소실을 방지 (스킵 커넥션)
        x = self.relu(x)

        return x

입력 -> { 기본블록 + 평균풀링 } (3번 반복) -> 평탄화 -> 분류기 -> 출력
        

In [ ]:
#@ ResNet 모델
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()

        # 기본 블록
        self.b1 = BasicBlock(in_channels=3, out_channels=64)
        self.b2 = BasicBlock(in_channels=64, out_channels=128)
        self.b3 = BasicBlock(in_channels=128, out_channels=256)
        # ((---) 256. 4. 4.) => (...) 256 * 4 * 4 = 4096

        # 평균 풀링 (<=>최대 풀링)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2) # 2x2 = 4.

        # 분류기
        # fully connected = MNP
        # https://blog.naver.com/PostView.nhn?blogId=intelliz&logNo=221709190464
        self.fc1 = nn.Linear(in_features=4096, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

        self.relu = nn.ReLU()
    
    # 순전파
    def forward(self, x):
        # 기본 블록과 풀링층 통과
        x = self.b1(x)
        x = self.pool(x) # 평균 풀
        x = self.b2(x)
        x = self.pool(x)
        x = self.b3(x)
        x = self.pool(x) # (-1, 256, 4, 4)
        print(x.shape)
        # 평탄화
        x = torch.flatten(x, start_dim=1)

        # 분류기
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x) # 활성화함수 X -> 출력층
        return x

In [ ]:
from tqdm.notebook import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor
from torchvision.transforms import RandomHorizontalFlip, RandomCrop
from torchvision.transforms import Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

transforms = Compose([
    RandomCrop((32, 32), padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

In [ ]:
from torchvision.datasets.utils import T
training_data = CIFAR10(root='./', train=True, download=True, transform=transforms)
test_data = CIFAR10(root='./', train=False, download=True, transform=transforms)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [ ]:
device = "cuda" if torch.cuda.is_available else "cpu"

model = ResNet(num_classes=10)
model.to(device)

ResNet(
  (b1): BasicBlock(
    (c1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasicBlock(
    (c1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasicBlock(
    (c1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
from torch.cuda import is_available
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)

for epoch in range(30):
    iterator = tqdm(train_loader)
    for data, label in iterator:
        optim.zero_grad()
        preds = model(data.to(device)) # 예측값
        # 손실 계산 및 역전파
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward() # 역전파
        optim.step() # 가중치 업데이트

        iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "ResNet.pth")

  0%|          | 0/1563 [00:00<?, ?it/s]

torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256,

  0%|          | 0/1563 [00:00<?, ?it/s]

torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256,

  0%|          | 0/1563 [00:00<?, ?it/s]

...

torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256,

  0%|          | 0/1563 [00:00<?, ?it/s]

torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256,

  0%|          | 0/1563 [00:00<?, ?it/s]

torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256,

In [ ]:
# 성능 평가하기
model.load_state_dict(torch.load('ResNet.pth', map_location=device))

num_corr = 0

with torch.no_grad():
    for data, label in test_loader:

        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr

    print(f"Accuracy: {num_corr/len(test_data)}")

torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256, 4, 4])
torch.Size([32, 256,

In [ ]:
# CNN 0.83 vs ResNet 0.88 vs VGG 0.92(사전학습)